# Crawling [bezdim.org](https://bezdim.org/signali/reports)

### Import Scrapy and install if missing

In [1]:
try:
    import scrapy
except:
    import sys
    !conda install --yes --prefix {sys.prefix} -c conda-forge scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess

Solving environment: done

## Package Plan ##

  environment location: /home/boris/anaconda3

  added / updated specs:
    - scrapy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    appdirs-1.4.3              |             py_1          11 KB  conda-forge
    automat-0.7.0              |             py_1          28 KB  conda-forge
    ca-certificates-2018.11.29 |       ha4d7672_0         143 KB  conda-forge
    certifi-2018.11.29         |        py37_1000         145 KB  conda-forge
    conda-4.6.3                |           py37_0         871 KB  conda-forge
    constantly-15.1.0          |             py_0           9 KB  conda-forge
    cssselect-1.0.3            |             py_0          16 KB  conda-forge
    hyperlink-17.3.1           |             py_0          28 KB  conda-forge
    incremental-17.5.0         |             py_0          14 KB  conda-forge
    openssl-1.1.1a  

#### Other imports

In [2]:
from time import time
from os.path import join
import pandas as pd

timestamp = int(time())

Currently not using the custom pipeline

In [3]:
import json
import codecs


class JsonWriterPipeline:

    def open_spider(self, spider):
        self.file = codecs.open('reportresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

### Helper functions

In [4]:
class ParseUtils:

    @staticmethod
    def parse_description(raw_description):
        striped = [d.strip() for d in raw_description if d.strip()]
        return " ".join(striped)
    
    @staticmethod
    def parse_time_location(when_where_element):
        selectors = when_where_element.xpath('.//span')
        if len(selectors) < 1:
            return ('N/A Data', 'N/A Location')
        
        date = selectors[0].css('span.r_date::text').extract_first(default='N/A Date')
        location = 'N/A Location'
        if len(selectors) > 1:
            location = selectors[1].css('span.r_location::text').extract_first(default='N/A Location')
    
        return date, location
    
    @staticmethod
    def parse_categories(categories_element):
        '''
        Can have multiple categories
        https://bezdim.org/signali/reports/view/10099
        '''
        selector = categories_element.xpath('.//p//a')
        categories = selector.css('a::text').extract()
        
        return [cat.strip() for cat in categories]
    
    @staticmethod
    def parse_file_urls(content_element):
        selector = content_element.xpath('.//ul//li')
        
        files = []
        
        for tag in selector.css('a'):
            url = tag.xpath('@href').extract_first(default='N/A url')
            name = tag.css('::text').extract_first(default='N/A filename')
            files.append({'url':url, 'name':name})
        
        return files

In [5]:
import logging
from scrapy import Spider
from scrapy.http import Request

class BezDimSpider(Spider):
    name = "bezdim"
    start_page = 1
    page_count = 296
    start_urls = [
        'https://bezdim.org/signali/reports'
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
#         'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},
        'FEED_FORMAT':'json',
        'FEED_URI': join('reports_data','report-result-{timestamp}.json'.format(timestamp=timestamp))
    }
    
    def start_requests(self):
        for i in range(self.start_page, self.start_page + self.page_count):
            yield Request('{url}/fetch_reports?page={page_id}'.
                          format(url=self.start_urls[0], page_id=i),
                    callback=self.process_page)
    
    def process_page(self, response):
        for link in response.css('a.r_title'):
            url = link.xpath('@href').extract_first()
            yield Request(url, callback=self.parse)
    
    def parse(self, report):
        report_id = int(report.url.split('/')[-1])
        title = report.css('h1.report-title::text').extract_first(default='').strip()
        
        raw_desc = report.css('div.report-description-text::text').extract()
        description = ParseUtils.parse_description(raw_desc)
        
        date, location = ParseUtils.parse_time_location(report.css('p.report-when-where'))
        
        categories = ParseUtils.parse_categories(report.css('div.report-category-list'))
        
        files = ParseUtils.parse_file_urls(report.css('div.content'))
        
        yield {
            'id': report_id,
            'title': title,
            'description': description,
            'date': date,
            'location': location,
            'categories': categories,
            'files': files
        }
        
#             description = report.css('div.r_description::text').extract_first().strip()
#             address = report.css('p.bd_location::text').extract_first().strip()
            
            

In [6]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(BezDimSpider)
process.start()

2019-02-08 19:19:42 [scrapy.utils.log] INFO: Scrapy 1.5.2 started (bot: scrapybot)
2019-02-08 19:19:42 [scrapy.utils.log] INFO: Versions: lxml 4.2.5.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 18.9.0, Python 3.7.1 (default, Dec 14 2018, 19:28:38) - [GCC 7.3.0], pyOpenSSL 18.0.0 (OpenSSL 1.1.1a  20 Nov 2018), cryptography 2.4.2, Platform Linux-4.19.16-1-MANJARO-x86_64-with-arch-Manjaro-Linux
2019-02-08 19:19:42 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'reports_data/report-result-1549646382.json', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


In [7]:
import pandas as pd

df = pd.read_json(
    join('reports_data','report-result-{timestamp}.json'.format(timestamp=timestamp)))

In [8]:
df

,categories,date,description,files,id,location,title
0,[заведение за хранене и развлечение],10:34 Jan 10 2019,"В момента се намирам в заведение ""Бакарди"" в г...",[],10533,"град Бургас, ул. Александровска № 51","Пушене в заведение ""Бакарди"", град Бургас"
1,[заведение за хранене и развлечение],08:31 Jan 10 2019,Не съм сигурен вече за кой пореден път подавам...,[{'url': 'https://bezdim.org/signali/media/upl...,10536,"град София, ж.к Гео Милев, Зала Фестивална","Пушене в заведение, пореден път, град София"
2,[заведение за хранене и развлечение],09:30 Mar 30 2018,"В сладкарница Малинка, пушат навсякъде! * Сигн...",[{'url': 'https://bezdim.org/signali/media/upl...,10244,"град София, кв. Младост, сп. Окръжна болница, ...","Пушене в сладкарница Малинка, град София"
3,[заведение за хранене и развлечение],02:13 Mar 31 2018,В Глори бар и грил се пуши дори през деня. * С...,[{'url': 'https://bezdim.org/signali/media/upl...,10249,"град София, ул. Пирински проход 24 А","Неспазване на забраната за тютюнопушене, град ..."
4,[заведение за хранене и развлечение],11:00 Jul 27 2018,име обект: Камино вид обект: пиано бар град: С...,[{'url': 'https://bezdim.org/signali/media/upl...,10362,"град София , ул. Неофит Рилски № 70","Пушене в пиано бар ""Камино"", град София (м)"
5,[заведение за хранене и развлечение],04:57 Apr 1 2018,"Здравейте, Моля за Вашето съдействие за взиман...",[],10252,"град Перник, ул. „Отец Паисий“ 8",Тютюнопушене и липса на климатизация в механа ...
6,[заведение за хранене и развлечение],00:43 Nov 11 2018,"В ресторант Родопска къща в Студентски град, С...",[{'url': 'https://bezdim.org/signali/media/upl...,10412,"град София, ул. ""8-ми декември"", блок 21","Пушене в ресторант Родопска къща, град София"
7,[заведение за хранене и развлечение],11:00 Feb 23 2018,име обект: Рико-33 ЕООД вид обект: Кафе-аперит...,[{'url': 'https://bezdim.org/signali/media/upl...,10173,"село Ракита, област Плевен, ул. ”Георги Димитр...","Пушене в кафе-аперитив/магазин ""Рико-33 ЕООД"",..."
8,[заведение за хранене и развлечение],09:37 Oct 10 2018,"Днес от 19:00 до 20:30 в заведението ""Дъ ривър...",[{'url': 'https://bezdim.org/signali/media/upl...,10395,"град Русе, ул. Чавдар Войвода 33 /пресечна с б...",нарушение на действащата забрана за тютюнопуше...
9,[заведение за хранене и развлечение],08:33 Nov 26 2017,В събота 18.11.2017 след 02:00ч в клуба се пуш...,[{'url': 'https://bezdim.org/signali/media/upl...,9936,"град София, ул. ""Георги С. Раковски"" 113","Пушене в Dirty Sofia, град София"
